In [3]:
import numpy as np
import pandas as pd
import scipy.stats as stats

# --- Synthetic data to illustrate a homogeneity (balance) check ---
np.random.seed(1)
n = 10_000

# 50/50 random assignment (ideal). Feel free to tweak the p vector to force an SRM.
variant = np.random.choice(["A", "B"], size=n, p=[0.5, 0.5])

# Pretend these are important pre‑experiment covariates
age = np.random.normal(loc=35, scale=10, size=n)          # Numeric
age[variant == "B"] += 1                                   # small imbalance

premium = np.random.binomial(1, p=np.where(variant == "A", 0.30, 0.32), size=n)  # Binary
baseline_metric = np.random.poisson(lam=np.where(variant == "A", 5, 5.1), size=n)  # Count

df = pd.DataFrame({
    "variant": variant,
    "age": age,
    "premium_user": premium,
    "baseline_metric": baseline_metric
})

# 1) Sample‑Ratio‑Mismatch (SRM) — chi‑squared test on counts
counts = df["variant"].value_counts().reindex(["A", "B"])
chi2_srm, p_srm = stats.chisquare(f_obs=counts, f_exp=[n/2, n/2])

# 2) Covariate balance — Standardized Mean Difference (SMD) + p‑values
rows = []
for col in ["age", "premium_user", "baseline_metric"]:
    A = df.loc[df.variant == "A", col]
    B = df.loc[df.variant == "B", col]

    # SMD (always on the pooled SD to keep scale‑free)
    smd = (A.mean() - B.mean()) / np.sqrt((A.var(ddof=1) + B.var(ddof=1)) / 2)

    # Quick frequentist test — Welch's t for numeric, chi‑square for binary
    if col == "premium_user":
        contingency = np.array([[A.sum(), len(A) - A.sum()],
                                [B.sum(), len(B) - B.sum()]])
        chi2, p = stats.chi2_contingency(contingency, correction=False)[:2]
    else:
        t, p = stats.ttest_ind(A, B, equal_var=False)

    rows.append({
        "covariate": col,
        "mean_A": round(A.mean(), 3),
        "mean_B": round(B.mean(), 3),
        "SMD": round(smd, 3),
        "p_value": round(p, 4)
    })

report = pd.DataFrame(rows)

print(f"SRM check — p‑value = {p_srm:.4f}")


SRM check — p‑value = 0.8415


In [4]:
report

,covariate,mean_A,mean_B,SMD,p_value
0,age,35.171,36.150,-0.098,0.0000
1,premium_user,0.298,0.321,-0.050,0.0123
2,baseline_metric,4.976,5.035,-0.026,0.1880
